Special Routines are available for evolving under a diagonal Coulomb operator.  This notebook describes how to use these built in routines and how they work.

In [1]:
from itertools import product
import fqe
from fqe.hamiltonians.diagonal_coulomb import DiagonalCoulomb

import numpy as np

import openfermion as of

from scipy.linalg import expm

In [2]:
#Utility function
def uncompress_tei(tei_mat, notation='chemistry'):
    """
    uncompress chemist notation integrals

    tei_tensor[i, k, j, l] = tei_mat[(i, j), (k, l)]
    [1, 1, 2, 2] = [1, 1, 2, 2] = [1, 1, 2, 2]  = [1, 1, 2, 2]
    [i, j, k, l] = [k, l, i, j] = [j, i, l, k]* = [l, k, j, i]*

    For real we also have swap of i <> j and k <> l
    [j, i, k, l] = [l, k, i, j] = [i, j, l, k] = [k, l, j, i]

    tei_mat[(i, j), (k, l)] = int dr1 int dr2 phi_i(dr1) phi_j(dr1) O(r12) phi_k(dr1) phi_l(dr1)

    Physics notation is the notation that is used in FQE.

    Args:
        tei_mat: compressed two electron integral matrix

    Returns:
        uncompressed 4-electron integral tensor. No antisymmetry.
    """
    if notation not in ['chemistry', 'physics']:
        return ValueError("notation can be [chemistry, physics]")

    norbs = int(0.5 * (np.sqrt(8 * tei_mat.shape[0] + 1) - 1))
    basis = {}
    cnt = 0
    for i, j in product(range(norbs), repeat=2):
        if i >= j:
            basis[(i, j)] = cnt
            cnt += 1

    tei_tensor = np.zeros((norbs, norbs, norbs, norbs))
    for i, j, k, l in product(range(norbs), repeat=4):
        if i >= j and k >= l:
            tei_tensor[i, j, k, l] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[k, l, i, j] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[j, i, l, k] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[l, k, j, i] = tei_mat[basis[(i, j)], basis[(k, l)]]

            tei_tensor[j, i, k, l] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[l, k, i, j] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[i, j, l, k] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[k, l, j, i] = tei_mat[basis[(i, j)], basis[(k, l)]]

    if notation == 'chemistry':
        return tei_tensor
    elif notation == 'physics':
        return np.asarray(tei_tensor.transpose(0, 2, 1, 3), order='C')

    return tei_tensor


The first example we will perform is diagonal Coulomb evolution on the Hartree-Fock state.  The diagonal Coulomb operator is defined as

\begin{align}
V = \sum_{\alpha, \beta \in \{\uparrow, \downarrow\}}\sum_{p,q} V_{pq,pq}n_{p,\alpha}n_{q,\beta}
\end{align}

The number of free parpameters are $\mathcal{O}(N^{2})$ where $N$ is the rank of the spatial basis. The `DiagonalCoulomb` Hamiltonian takes either a generic 4-index tensor or the $N \times N$ matrix defining $V$.  If the 4-index tensor is given the $N \times N$ matrix is constructed along with the diagonal correction.  If the goal is to just evolve under $V$ it is recommended the user input the $N \times N$ matrix directly.

All the terms in $V$ commute and thus we can evolve under $V$ exactly by counting the accumulated phase on each bitstring.


To start out let's define a Hartree-Fock wavefunction for 4-orbitals and 2-electrons $S_{z} =0$.

In [3]:
norbs = 4
tedim = norbs * (norbs + 1) // 2
if (norbs // 2) % 2 == 0:
    n_elec = norbs // 2
else:
    n_elec = (norbs // 2) + 1
sz = 0
fqe_wfn = fqe.Wavefunction([[n_elec, sz, norbs]])
fci_data = fqe_wfn.sector((n_elec, sz))
fci_graph = fci_data.get_fcigraph()
hf_wf = np.zeros((fci_data.lena(), fci_data.lenb()), dtype=np.complex128)
hf_wf[0, 0] = 1  # right most bit is zero orbital.
fqe_wfn.set_wfn(strategy='from_data',
                raw_data={(n_elec, sz): hf_wf})
fqe_wfn.print_wfn()

Sector N = 2 : S_z = 0
a'0001'b'0001' (1+0j)


Now we can define a random 2-electron operator $V$.  To define $V$ we need a $4 \times 4$ matrix.  We will generate this matrix by making a full random two-electron integral matrix and then just take the diagonal elements

In [4]:
tei_compressed = np.random.randn(tedim**2).reshape((tedim, tedim))
tei_compressed = 0.5 * (tei_compressed + tei_compressed.T)
tei_tensor = uncompress_tei(tei_compressed, notation='physics')

diagonal_coulomb = of.FermionOperator()
diagonal_coulomb_mat = np.zeros((norbs, norbs))
for i, j in product(range(norbs), repeat=2):
    diagonal_coulomb_mat[i, j] = tei_tensor[i, j, i, j]
    for sigma, tau in product(range(2), repeat=2):
        diagonal_coulomb += of.FermionOperator(
            ((2 * i + sigma, 1), (2 * i + sigma, 0), (2 * j + tau, 1),
             (2 * j + tau, 0)), coefficient=diagonal_coulomb_mat[i, j])

dc_ham = DiagonalCoulomb(diagonal_coulomb_mat)


Evolution under $V$ can be computed by looking at each bitstring, seeing if $n_{p\alpha}n_{q\beta}$ is non-zero and then phasing that string by $V_{pq}$.  For the Hartree-Fock state we can easily calculate this phase accumulation.  The alpha and beta bitstrings are "0001" and "0001". 

In [5]:
alpha_occs = [list(range(fci_graph.nalpha()))]
beta_occs = [list(range(fci_graph.nbeta()))]
occs = alpha_occs[0] + beta_occs[0]
diag_ele = 0.
for ind in occs:
    for jnd in occs:
        diag_ele += diagonal_coulomb_mat[ind, jnd]
evolved_phase = np.exp(-1j * diag_ele)
print(evolved_phase)

# evolve FQE wavefunction
evolved_hf_wfn = fqe_wfn.time_evolve(1, dc_ham)

# check they the accumulated phase is equivalent!
assert np.isclose(evolved_hf_wfn.get_coeff((n_elec, sz))[0, 0], evolved_phase)


(-0.9820421521244929-0.18866163216376114j)


We can now try this out for more than 2 electrons.  Let's reinitialize a wavefunction on 6-orbitals with 4-electrons $S_{z} = 0$ to a random state.

In [6]:
norbs = 6
tedim = norbs * (norbs + 1) // 2
if (norbs // 2) % 2 == 0:
    n_elec = norbs // 2
else:
    n_elec = (norbs // 2) + 1
sz = 0
fqe_wfn = fqe.Wavefunction([[n_elec, sz, norbs]])
fqe_wfn.set_wfn(strategy='random')
inital_coeffs = fqe_wfn.get_coeff((n_elec, sz)).copy()
print("Random initial wavefunction")
fqe_wfn.print_wfn()

Random initial wavefunction
Sector N = 4 : S_z = 0
a'000011'b'000011' (-0.01170366683048796-0.02611174638123927j)
a'000011'b'000101' (-0.03046291698061075-0.009647949016890195j)
a'000011'b'001001' (-0.008988018827580787+0.038596975665611814j)
a'000011'b'010001' (0.051853495234833985-0.03927812614298746j)
a'000011'b'100001' (0.03646006947356057+0.035480239335700464j)
a'000011'b'000110' (0.01880254338573725-0.0529751927248195j)
a'000011'b'001010' (0.0404618549945504+0.062203524093458314j)
a'000011'b'010010' (-0.0031844480852456524+0.016711839169505995j)
a'000011'b'100010' (0.009415807513407632-0.04125278712962207j)
a'000011'b'001100' (-0.027849189791025325+0.026399654806523707j)
a'000011'b'010100' (0.011510526895181977+0.018247865499874505j)
a'000011'b'100100' (-0.008236314403493128-0.0008755403241821855j)
a'000011'b'011000' (-0.06526550460215325+0.02136134327454697j)
a'000011'b'101000' (-0.04381949627920303-0.022754950858181685j)
a'000011'b'110000' (-0.11945291846412728-0.03859738862228

We need to build our Diagoanl Coulomb operator For this bigger system.

In [7]:
tei_compressed = np.random.randn(tedim**2).reshape((tedim, tedim))
tei_compressed = 0.5 * (tei_compressed + tei_compressed.T)
tei_tensor = uncompress_tei(tei_compressed, notation='physics')

diagonal_coulomb = of.FermionOperator()
diagonal_coulomb_mat = np.zeros((norbs, norbs))
for i, j in product(range(norbs), repeat=2):
    diagonal_coulomb_mat[i, j] = tei_tensor[i, j, i, j]
    for sigma, tau in product(range(2), repeat=2):
        diagonal_coulomb += of.FermionOperator(
            ((2 * i + sigma, 1), (2 * i + sigma, 0), (2 * j + tau, 1),
             (2 * j + tau, 0)), coefficient=diagonal_coulomb_mat[i, j])

dc_ham = DiagonalCoulomb(diagonal_coulomb_mat)


Now we can convert our wavefunction to a cirq wavefunction, evolve under the diagonal_coulomb operator we constructed and then compare the outputs.

In [8]:
cirq_wfn = fqe.to_cirq(fqe_wfn).reshape((-1, 1))
final_cirq_wfn = expm(-1j * of.get_sparse_operator(diagonal_coulomb)) @ cirq_wfn
# recover a fqe wavefunction
from_cirq_wfn = fqe.from_cirq(final_cirq_wfn.flatten(), 1.0E-8)


/Users/nickrubin/anaconda3/envs/fqe-dev/lib/python3.6/site-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [9]:
fqe_wfn = fqe_wfn.time_evolve(1, dc_ham)
print("Evolved wavefunction")
fqe_wfn.print_wfn()

Evolved wavefunction
Sector N = 4 : S_z = 0
a'000011'b'000011' (0.028430438135459493+0.0032418056362214145j)
a'000011'b'000101' (-0.018517508308895387-0.02604177638391985j)
a'000011'b'001001' (-0.03960246305915306+0.001468309444631671j)
a'000011'b'010001' (-0.04766025429709583-0.04427252332665875j)
a'000011'b'100001' (0.0073673649804231744+0.05033791794046198j)
a'000011'b'000110' (-0.055276455628545114+0.010218617085975174j)
a'000011'b'001010' (-0.07113758029674105+0.021115984200841556j)
a'000011'b'010010' (-0.00751291103779322+0.01526376250382796j)
a'000011'b'100010' (0.011058519660690068-0.040843102477720605j)
a'000011'b'001100' (-0.012543649992202675+0.03626535524151491j)
a'000011'b'010100' (-0.01485539445061667-0.01564589660015703j)
a'000011'b'100100' (0.0015612515835241362+0.008134244851576622j)
a'000011'b'011000' (-0.06267530629226217+0.028065976887289033j)
a'000011'b'101000' (0.04592947380490362+0.018122347494903178j)
a'000011'b'110000' (0.12431187722636629+0.01747327440733943j)

In [10]:
print("From Cirq Evolution")
from_cirq_wfn.print_wfn()
assert np.allclose(from_cirq_wfn.get_coeff((n_elec, sz)),
                   fqe_wfn.get_coeff((n_elec, sz)))
print("Wavefunctions are equivalent")

From Cirq Evolution
Sector N = 4 : S_z = 0
a'000011'b'000011' (0.028430438135459483+0.0032418056362214375j)
a'000011'b'000101' (-0.01851750830889539-0.026041776383919853j)
a'000011'b'001001' (-0.039602463059153055+0.0014683094446316367j)
a'000011'b'010001' (-0.047660254297095785-0.04427252332665877j)
a'000011'b'100001' (0.007367364980423216+0.05033791794046195j)
a'000011'b'000110' (-0.055276455628545114+0.010218617085975155j)
a'000011'b'001010' (-0.07113758029674108+0.021115984200841546j)
a'000011'b'010010' (-0.0075129110377932205+0.015263762503827964j)
a'000011'b'100010' (0.011058519660690071-0.040843102477720605j)
a'000011'b'001100' (-0.012543649992202678+0.03626535524151493j)
a'000011'b'010100' (-0.014855394450616675-0.01564589660015703j)
a'000011'b'100100' (0.001561251583524135+0.008134244851576624j)
a'000011'b'011000' (-0.06267530629226223+0.028065976887289054j)
a'000011'b'101000' (0.045929473804903614+0.018122347494903178j)
a'000011'b'110000' (0.12431187722636629+0.01747327440733

Finally, we can compare against evolving each term of $V$ individually.

In [11]:
fqe_wfn = fqe.Wavefunction([[n_elec, sz, norbs]])
fqe_wfn.set_wfn(strategy='from_data',
                raw_data={(n_elec, sz): inital_coeffs})
for term, coeff in diagonal_coulomb.terms.items():
    op = of.FermionOperator(term, coefficient=coeff)
    fqe_wfn = fqe_wfn.time_evolve(1, op)

assert np.allclose(from_cirq_wfn.get_coeff((n_elec, sz)),
               fqe_wfn.get_coeff((n_elec, sz)))
print("Individual term evolution is equivalent")

Individual term evolution is equivalent
